#### Importing necessary packages ####

In [164]:
import numpy as np;
import matplotlib.pyplot as pyplot;
import pandas as pd;
import sklearn;
import regex as re;
from sklearn.model_selection import train_test_split


In [159]:
df = pd.read_csv("used_cars.csv");
#verifying mission values
df['milage'].isnull().sum()

# replacing Null values with mode
df['fuel_type'].replace(np.nan,df['fuel_type'].mode()[0])

# df['clean_title'].replace(np.nan,df['clean_title'].mode()[0])
# dropping rows
df = df.dropna(subset = ['clean_title'])
# df['clean_title'].isnull().sum()

In [160]:
df = df.drop(columns = ['engine','model'])
# turning milage,price values into numbers
for i,row in df.iterrows():
    number = int("".join(re.findall("\d+",row['milage'])))
    df.loc[i,'milage'] = number;
    number = int("".join(re.findall("\d+",row['price'])))
    df.loc[i,'price'] = number;
    
df

,brand,model_year,milage,fuel_type,transmission,ext_col,int_col,accident,clean_title,price
0,Ford,2013,51000,E85 Flex Fuel,6-Speed A/T,Black,Black,At least 1 accident or damage reported,Yes,10300
1,Hyundai,2021,34742,Gasoline,8-Speed Automatic,Moonlight Cloud,Gray,At least 1 accident or damage reported,Yes,38005
3,INFINITI,2015,88900,Hybrid,7-Speed A/T,Black,Black,None reported,Yes,15500
6,Audi,2017,84000,Gasoline,6-Speed A/T,Blue,Black,None reported,Yes,31000
7,BMW,2001,242000,Gasoline,A/T,Green,Green,None reported,Yes,7300
...,...,...,...,...,...,...,...,...,...,...
4003,Mercedes-Benz,2018,53705,Gasoline,A/T,Black,Black,At least 1 accident or damage reported,Yes,25900
4004,Bentley,2023,714,Gasoline,8-Speed Automatic with Auto-Shift,C / C,Hotspur,None reported,Yes,349950
4005,Audi,2022,10900,Gasoline,Transmission w/Dual Shift Mode,Black,Black,None reported,Yes,53900
4007,Ford,2020,33000,Gasoline,A/T,Blue,Black,None reported,Yes,62999


In [161]:
# turning categorical columns into classes

def turn_into_class(df,col_name):

    mean_encoded = df.groupby(col_name)['price'].mean()

    df[col_name] = df[col_name].map(mean_encoded)
    

turn_into_class(df,col_name='brand');
turn_into_class(df,col_name='fuel_type');
turn_into_class(df,col_name='transmission');
turn_into_class(df,col_name='ext_col');
turn_into_class(df,col_name='int_col');
turn_into_class(df,col_name='clean_title');
turn_into_class(df,col_name='accident');
df
# extcolor_class_index = turn_into_class(df['ext_col'],col_name='ext_col');
# intcolor_class_index = turn_into_class(df['int_col'],col_name='int_col')
# cleantitle_class_index = turn_into_class(df['clean_title'],col_name='clean_title')
# accident_class_index = turn_into_class(df['accident'],col_name='accident')
# fuel_class_index = turn_into_class(df['fuel_type'],col_name='fuel_type')
# transmission_class_index = turn_into_class(df['transmission'],col_name='transmission')






,brand,model_year,milage,fuel_type,transmission,ext_col,int_col,accident,clean_title,price
0,34172.233618,2013,51000,22119.367188,25252.872881,39494.453125,41951.396463,28579.415873,41734.390272,10300
1,18292.666667,2021,34742,41610.462167,93972.935484,38005.0,26759.391403,28579.415873,41734.390272,38005
3,21796.148148,2015,88900,49021.324138,47571.348259,39494.453125,41951.396463,46771.444895,41734.390272,15500
6,34424.808917,2017,84000,41610.462167,25252.872881,38085.957317,41951.396463,46771.444895,41734.390272,31000
7,37542.046729,2001,242000,41610.462167,31200.125123,44600.014493,37014.142857,46771.444895,41734.390272,7300
...,...,...,...,...,...,...,...,...,...,...
4003,50676.660517,2018,53705,41610.462167,31200.125123,39494.453125,41951.396463,28579.415873,41734.390272,25900
4004,122850.111111,2023,714,41610.462167,215849.8,349950.0,294972.5,46771.444895,41734.390272,349950
4005,34424.808917,2022,10900,41610.462167,54031.565553,39494.453125,41951.396463,46771.444895,41734.390272,53900
4007,34172.233618,2020,33000,41610.462167,31200.125123,38085.957317,41951.396463,46771.444895,41734.390272,62999


In [166]:
train_x,test_x,train_y,test_y = train_test_split(df[['brand','model_year','milage','fuel_type','transmission','ext_col','int_col','accident','clean_title']], df['price'], test_size=0.2, random_state=42)

,brand,model_year,milage,fuel_type,transmission,ext_col,int_col,accident,clean_title
2460,32760.214286,2002,150000,41610.462167,14329.438596,39494.453125,26759.391403,28579.415873,41734.390272
3384,26003.03,2017,187141,41610.462167,31200.125123,39494.453125,41951.396463,46771.444895,41734.390272
2621,25921.909091,2003,142215,41610.462167,13772.513889,44478.030108,32965.225243,46771.444895,41734.390272
2625,34172.233618,2016,110450,41610.462167,25252.872881,32065.743516,45294.86755,46771.444895,41734.390272
2874,34172.233618,2022,4500,41610.462167,59995.765217,39494.453125,41951.396463,46771.444895,41734.390272
...,...,...,...,...,...,...,...,...,...
1377,29110.868421,2003,116157,41610.462167,13772.513889,27498.432432,32965.225243,46771.444895,41734.390272
1419,32760.214286,2003,114000,41610.462167,17485.47619,41463.747066,32965.225243,28579.415873,41734.390272
1607,21796.148148,2016,71000,41610.462167,31200.125123,39494.453125,41951.396463,46771.444895,41734.390272
1102,41574.930556,2021,22000,41610.462167,50530.294118,39494.453125,41951.396463,46771.444895,41734.390272
